In [1]:
from skimage import io,transform #scikit-image used for image processing
import glob                     #glob for serching a file path comlying to specific rule
import os                  #os provide large no of functions to process files and directories
import tensorflow as tf     # tf for developing DL algorithms and for constructing CNN
import numpy as np         #numpy array used as image object
import time                # time processes and converts time

#preparing paths
path='flower_photos/'
model_path='Model/model.ckpt'

#step3: preparing standard image parameters.w,h,c are width,lenght and depth of the image
w=100
h=100
c=3

#step4:performing data reading and standardization
def read_img(path):
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)]
    imgs=[]
    labels=[]
    for idx,folder in enumerate(cate):
        for im in glob.glob(folder+'/*.jpg'):
            print('reading the images:%s'%(im))
            img=io.imread(im)
            img=transform.resize(img,(w,h))
            imgs.append(img)
            labels.append(idx)
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)

data,label=read_img(path)
print("shape of data:",data.shape)
print("shape of label:",label.shape)




FileNotFoundError: [WinError 3] The system cannot find the path specified: 'flower_photos/'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data pre-processing


In [0]:
#step1: Disrupting original data sequence ,use shuffle
num_example=data.shape[0]
arr=np.arange(num_example)
np.random.shuffle(arr)
data=data(arr)
label=label[arr]


In [0]:
#step2: setting the training set and verification set
ratio=0.8
s=np.int(num_example*ratio)
x_train=data[:s]
y_train=label[:s]
x_val=data[s:]
y_val=label[s:]

# Placeholder defining

In [0]:
#creating placeholder x and y to perform model calculation for the CNN model to transmit tensor data
x=tf.placeholder(tf.float32,shape=[None,w,h,c],name='x')
y=tf.placeholder(tf.int32,shape=[None,],name='y_')

# CNN Construction


In [0]:
#step1:create a CNN and the hidden layer
#define CNN model. 3 varibales,input_tensor,train and regularizer
def inference(input_tensor, train, regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weights=tf.get_variable("weight",[5,5,3,32],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases=tf.get_variable("bias",[32], initializer=tf.constant_initializer(0.0))
        conv1=tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu1=tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

In [0]:
#Step2: creating second hidden layer
with tf.name_scope("layer2-pool1"):
    pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="VALID")

In [0]:
#step3:createthird hidden layer ie layer1-convo2
with tf.variable_scope("layer3-convo2"):
    convo2_weigths = tf.get_variable("weight",[5,5,32,64],initializer=tf.truncated_normal_initializer(stddev=0.1))
    convo2_biases = tf.get_variable("bias",[64], initializer=tf.constant_initializer(0.0))
    convo2 = tf.nn.convo2d(pool1, convo2_weights, strides=[1, 1, 1, 1], padding="SAME")
    relu2 = tf.nn.relu(tf.nn.bias_add(convo2, conv2_biases))


In [0]:
#step4:creating the fourth layer
with tf.name_scope("layer4-pool2"):
    pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding = "VALID")
    nodes = 25*25*64
    reshaped = tf.reshape(pool2, [-1,nodes])

In [0]:
#step4: creating the  fifth hidden layer
with tf.variable_scope("layer5-fc1"):
    fc1_weights = tf.get_variable("weight", [nodes, 1024],initializer=tf.truncated_normal_initializer(stddev=0.1))
    if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
    fc1_biases = tf.get_variable("bias", [1024], initializer=tf.constant_initializer(0.1))
    
    fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
    if train: fc1 = tf.nn.dropout(fc1, 0.05)
        

In [0]:
#step 6: create the sixth hidden layer
with tf.variable_scope('layer6-fc2'):
    fc2_weights = tf.get_variable("weight", [1024, 5], initializer=tf.truncated_normal_initializer(stddev=0.1))
    if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
    fc2_biases = tf.get_variable("bias", [5], initializer=tf.constant_initializer(0.1))
    logit = tf.matmul(fc1, fc2_weights) + fc2_biases

In [0]:
#step7: now we want to view the overarl network structure
#entire model has 6 hidden layers, two convolutionary layers,two pooling layers, and two full connection layers

def inference (input_tensor, train, regularizer):
    with tf.variable_scope('layer1-convo1'):
        convo1_weights = tf.get_variable("weights", [5,5,3,32], initializer= tf.truncated_normal_initializer(stddev=0.1))
        convo1_biases = tf.get_variable("bias", [32], initializer=tf.constant_initializer(0.0))
        convo1 = tf.nn.convo2d(input_tensor, convo1_weights,strides=[1, 1, 1, 1], paddding = 'SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(convo1, convo1_biases))
        
    with tf.namescope("layer2-pool1"):
        pool1 = tf.nn.max_pool(relu1, ksize= [1,2,2,1], strides=[1,2,2,1], padding="VALID")
    
    with tf.variable_scope("layer3-convo2"):
        convo2_weights = tf.get_varible("weight",[5,5,32,64],initializer=tf.truncated_normal_initializer(stddev=0.1))
        convo2_biases = tf.get_variable("bias", [64], initializer=tf.constant_initializer(0.0))
        convo2 = tf.nn.convo2d(pool1, convo2_weights,strides=[1, 1, 1, 1], paddding = 'SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(convo2, convo2_biases))
    
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize= [1,2,2,1], strides=[1,2,2,1], padding="VALID")
        nodes = 25*25*64
        reshaped = tf.reshape(pool2,[-1,nodes])
    
    with tf.variable_scope('layer5-fc1'):
        fc1_weights = tf.get_variable("weight", [nodes,1024], initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias", [1024], initializer=tf.constant_initializer(0.1))
        
        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        tf train: fc1 = tf.nn.dropout(fc1, 0.5)
    
     with tf.variable_scope('layer6-fc2'):
        fc2_weights = tf.get_variable("weight", [1024, 5], initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias", [5], initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases
    
    return logit
        

In [0]:
#step8: defining
# the document mainly uses L2  regular terms to prevent  overfitting and improve model generalization capabilities

regularizer = tf.contrib.layers.12_regularizer(0.001)


In [0]:
#step 9 : viewing the model, to prepare subsequent modeltest

logits = model(x, False, regularizer)
print("shape of logits:",logits.shape)

b = tf.constant(value=1, dtype=tf.float32)
logits_eval = tf.multiply(logits,b,name='logits_eval')

# Loss function, optimizer, and verification indicator defining

In [0]:
#step1: defining the loss function
#calculating the sparse cross entropy between the prediction tag logits and original tag y_

loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_)

In [0]:
#step2:defining optimizer, using adam optimizer

optimizer = tf.train.AdamOptimizer(Learning_rate=0.001)

In [0]:
#step3: defining optimization target, training operation and minimizing the loss

train_op = optimizer.minimize(loss)

In [0]:
#step4: defining accuracy of verification indicators.accuracay function for model effect verification

correct_prediction = tf.equal(tf.cast(tf.argmax(logits,1), tf.int32), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# Model Training and Verification

In [0]:
#step1: defining the function for extracting data in batches

def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt], targets[excerpt]

In [0]:
#step 2 :prepare training parameters and perform model initialization

n_epoch = 10
batch_size = 64

saver = tf.train.Saver()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
#step3: defining model operation on the training set and verification set and print the results
#training a total of 5 epochs, each epoch need to run on the training set and verification set
#print the corresponding loss value

for epoch in range(n_epoch):
    print("epooch", epoch+1)
    start_time = time.time()
    
    #training
    train_loss, train_acc, n_batch = 0, 0, 0
    for x_train_a, y_train_a in minibatches(x_train, y_train, batch_size, shuffle=True):
        _,err,ac=sess.run([train_op,loss,acc], feed_dict={x: x_train_a, y_:y_train_a})
        train_loss += err; train_acc += ac; n_batch +=1
    print(" train loss: %f" % (np.sum(train_loss)/n_batch))
    print(" train acc: %f" % (np.sum(train_acc)/n_batch))
    
    #validation
    val_loss, val_acc, n_batch = 0, 0, 0
    for x_val_a, y_val_a in minibatches(x_train, y_train, batch_size, shuffle=False):
        err, ac = sess.run([train_op, loss, acc], feed_dict={x: x_train_a, y_:y_train_a})
        val_loss += err; val_acc += ac; n_batch += 1
    print(" validation loss: %f" % (np.sum(val_loss)/n_batch))
    print(" validation acc: %f" % (np.sum(val_acc)/n_batch))
    print(" epoch time: %f" % (time.time() - start_time)
    print('--------------------------------------------------------')

    
        
    

# Model saving and session closing

In [0]:
#step1: saving the model,

saver.save(sess,model_path)

In [0]:
#step2: closing the session
sess.close()


# Sample image testing, invoking the Model for testing

In [0]:
#step1: import modules
path1 = "flower_photos/daisy/5547758_eea9edfd54_n.jpg"
path2 = "flower_photos/dandelion/735522_b66e5d3078_m.jpg"
path3 = "flower_photos/roses/394990940_7af082cf8d_n.jpg"
path4 = "flower_photos/sunflowers/6953297_8576bf4ea3.jpg"
path5 = "flower_photos/tulips/10791227_7168491604.jpg"


In [0]:
#step2: generate a type dictionary, for each flower type for type matching
flower_dict = {0:'daisy',1:'dandelion',2:'roses',3:'sunflowers',4:'tulips'}

In [0]:
#step3: defining image starndardization function
w=100
h=100
c=3

def read_one_image(path):
    img = io.imread(path)
    img = transform.resize(img, (w,h))
    return np.asarray(img)

In [0]:
#step 4: standard the test data, invoke the read one image

with tf.Session() as sess:
    data = []
    data1 = read_one_image(path1)
    data2 = read_one_image(path2)
    data3 = read_one_image(path3)
    data4 = read_one_image(path4)
    data5 = read_one_image(path5)
    data.append(data1)
    data.append(data2)
    data.append(data3)
    data.append(data4)
    data.append(data5)


In [0]:
#step5: Reload the model
saver = tf.train.import_meta_graph('Model/model.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('Model'))

In [0]:
#step 6: output model parameters
graph = tf.get_default_graph()
x = graph.get_tensor_by_name("x:0")

In [0]:
#step 7: invoking the model
feed_dict = {x:data}

logits = graph.get_tensor_by_name("logits_eval:0")

classifcation_result = sess.run(logits, feed_dict)

print(classification_result)
print(tf.argmax(classification_result,1).eval())

In [0]:
#step 8: print the test resulst
output = []
output = tf.argmax(classification_result, 1).eval()
for i in range(len(output)):
    print("flower", i+1,"prediction:"+flower_dict[output[i]])


NameError: name 'tf' is not defined